<a href="https://colab.research.google.com/github/jinyaoz/yelpScoresPredictor/blob/master/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Sentiment Analysis using NLTK, regular expression, and pyTorch LSTM
some snippets are extracted from https://github.com/gabrielloye/LSTM_Sentiment-Analysis/blob/master/main.ipynb

# Intro

In this file, we perform sentiment analysis of texts using Long Short-Term Memory Network (LSTM) from pyTorch. 

In [0]:
from collections import Counter
import re
import nltk
import numpy as np

# Preprocessing

In [0]:
import pandas as pd

class data:
    def __init__(self, fileName):
        df = pd.read_csv(fileName)
        values = df.values
        # Star ratings, notice for test/validation data, these are actually ids
        self.stars = np.array(values[:, 0], dtype = int)
        # Name of restaurants
        self.names = values[:, 1]
        # Text of review
        self.text = values[:, 2]
        # Date of review
        self.date = values[:, 3]
        # City of restaurant
        self.city = values[:, 7]
        # Misc categories, might not use it
        self.category = values[:, 10]
        # Sentiment score
        self.sentiment = np.nan_to_num(values[:, 13])
        s = {0, 1, 2, 3, 7, 10}
        cols = []
        for i in range(values.shape[1]):
            if i not in s: cols.append(i)
        # Other numerical measurements
        self.numerical = np.array(values[:, cols], dtype = np.float64)
        self.text_df = None


train_data = data("/content/Yelp_train.csv")
validation_data = data("/content/Yelp_validate.csv")
test_data = data("/content/Yelp_test.csv")

# instantiate the training texts, validation texts, and testing texts
train_txt = list(train_data.text)
test_txt = test_data.text

train_labels = train_data.stars



now we make a dictionary for all the words appeared in the texts, this would be our vocabulary

In [20]:
# standard cleaning of sentences
for i in range(len(train_txt)):
    if i == 0: continue
    #print(train_txt[1])
    train_txt[i] = re.sub('\d','0',train_txt[i])

for i in range(len(test_txt)):
    test_txt[i] = re.sub('\d','0',test_txt[i])

words = Counter() #Dictionary that will map a word to the number of times it appeared in all the training sentences
for i, sentence in enumerate(train_txt):
    #The sentences will be stored as a list of words/tokens
    if i == 0: continue
    print(sentence)
    word_list = sentence.split(" ")
    train_txt[i] = []
    for word in nltk.word_tokenize(word_list): #Tokenizing the words
        words.update([word.lower()]) #Converting all the words to lower case
        train_txt[i].append(word)


TypeError: ignored

To remove typos and words that likely don't exist, we'll remove all words from the vocab that only appear once throughout. To account for unknown words and padding, we'll have to add them to our vocabulary as well. Each word in the vocabulary will then be assigned an integer index and thereafter mapped to this integer.

In [0]:
# Removing the words that only appear once
words = {k:v for k,v in words.items() if v>1}
# Sorting the words according to the number of appearances, with the most common word being first
words = sorted(words, key=words.get, reverse=True)
# Adding padding and unknown to our vocabulary so that they will be assigned an index
words = ['_PAD','_UNK'] + words
# Dictionaries to store the word to index mappings and vice versa
word2idx = {o:i for i,o in enumerate(words)}
idx2word = {i:o for i,o in enumerate(words)}

# convert the sentences into their corresponding indices
for i, sentence in enumerate(train_txt):
    # Looking up the mapping dictionary and assigning the index to the respective words
    train_txt[i] = [word2idx[word] if word in word2idx else 0 for word in sentence]

for i, sentence in enumerate(test_txt):
    # For test sentences, we have to tokenize the sentences as well
    test_txt[i] = [word2idx[word.lower()] if word.lower() in word2idx else 0 for word in nltk.word_tokenize(sentence)]


finally, we define a function for sentence padding and process the sentences

In [0]:
def pad_input(sentences, seq_len):
    features = np.zeros((len(sentences), seq_len),dtype=int)
    for ii, review in enumerate(sentences):
        if len(review) != 0:
            features[ii, -len(review):] = np.array(review)[:seq_len]
    return features

seq_len = 200 #The length that the sentences will be padded/shortened to

train_txt = pad_input(train_txt, seq_len)
test_txt = pad_input(test_txt, seq_len)



# Setting up LSTM
In this section, we will split the labelled data to training data and validation data, and then set up the pyTorch tensors for LSTM

In [0]:
# splitting the training and validation data 8:2 using train_test split from sklearn
from sklearn.model_selection import train_test_split

train_sentences, train_labels, val_sentences, val_labels = train_test_split(train_txts, train_labels, random_state = 2019
                                                                                          test_size = 0.2)
train_data = TensorDataset(torch.from_numpy(train_sentences), torch.from_numpy(train_labels))
val_data = TensorDataset(torch.from_numpy(val_sentences), torch.from_numpy(val_labels))

train_data = TensorDataset(torch.from_numpy(train_sentences), torch.from_numpy(train_labels))
val_data = TensorDataset(torch.from_numpy(val_sentences), torch.from_numpy(val_labels))
#test_data = TensorDataset(torch.from_numpy(test_sentences), torch.from_numpy(test_labels))

batch_size = 400

train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
val_loader = DataLoader(val_data, shuffle=True, batch_size=batch_size)
#test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)

# checking if GPU is available

# torch.cuda.is_available() checks and returns a Boolean True if a GPU is available, else it'll return False
is_cuda = torch.cuda.is_available()

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")


# LSTM Modelling
In this section, we are defining our own model for LSTM. This model is tentative.

In [0]:
import torch.nn as nn

class SentimentNet(nn.Module):
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):
        super(SentimentNet, self).__init__()
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=drop_prob, batch_first=True)
        self.dropout = nn.Dropout(0.2)
        self.fc = nn.Linear(hidden_dim, output_size)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x, hidden):
        batch_size = x.size(0)
        x = x.long()
        embeds = self.embedding(x)
        lstm_out, hidden = self.lstm(embeds, hidden)
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
        out = self.dropout(lstm_out)
        out = self.fc(out)
        out = self.sigmoid(out)
        
        out = out.view(batch_size, -1)
        out = out[:,-1]
        return out, hidden
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device))
        return hidden

With this, we can instantiate our model after defining the arguments. The output dimension will only be 1 as it only needs to output 1 or 0. The learning rate, loss function and optimizer are defined as well.

In [0]:
vocab_size = len(word2idx) + 1
output_size = 1
embedding_dim = 400
hidden_dim = 512
n_layers = 2

model = SentimentNet(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)
model.to(device)
print(model)

lr=0.005
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [0]:

epochs = 2
counter = 0
print_every = 1000
clip = 5
valid_loss_min = np.Inf

model.train()
for i in range(epochs):
    h = model.init_hidden(batch_size)
    
    for inputs, labels in train_loader:
        counter += 1
        h = tuple([e.data for e in h])
        inputs, labels = inputs.to(device), labels.to(device)
        model.zero_grad()
        output, h = model(inputs, h)
        loss = criterion(output.squeeze(), labels.float())
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        
        if counter%print_every == 0:
            val_h = model.init_hidden(batch_size)
            val_losses = []
            model.eval()
            for inp, lab in val_loader:
                val_h = tuple([each.data for each in val_h])
                inp, lab = inp.to(device), lab.to(device)
                out, val_h = model(inp, val_h)
                val_loss = criterion(out.squeeze(), lab.float())
                val_losses.append(val_loss.item())
                
            model.train()
            print("Epoch: {}/{}...".format(i+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))
            if np.mean(val_losses) <= valid_loss_min:
                torch.save(model.state_dict(), './state_dict.pt')
                print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,np.mean(val_losses)))
                valid_loss_min = np.mean(val_losses)